### Env Config

In [1]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [2]:
# Aproximação da quantidade de linhas esperadas no arquivo final 
TOTAL_ROW_COUNT = 1000000
# Randomizar amostras coletadas de cada arquivo
DO_RANDOMIZE_SAMPLE_ROWS = True

# Exportar os arquivos tratados ou não
DO_EXPORT_FILES = True
# Tipo de arquivo a ser gerado
OUTPUT_FILE_TYPE = 'csv' # 'csv' ou 'parquet'

# Rodar ou não a visualização dos recursos (Demora muito mais tempo)
RUN_RESOURCE_VISUALIZATION = False

In [3]:
def export_to_file(df: DataFrame, path: str):
	if DO_EXPORT_FILES:
		if not os.path.exists(path):
			os.makedirs(path)
		if OUTPUT_FILE_TYPE == 'csv':
			# df.write.csv(f'{path}/df.csv', header=True, mode='overwrite')
			df.to_csv(f'{path}/df.csv', index=False)
		else:
			# df.write.parquet(f'{path}/df.parquet', mode='overwrite')
			df.to_parquet(f'{path}/df.parquet', index=False)

### Criando dataframe

In [4]:
files = os.listdir('_data/raw/operacoes-credito')
files.sort()

sample_size = TOTAL_ROW_COUNT // len(files)

arrow_tables = []
for file in files:
	print(f'Processando o arquivo {file}')

	temp_df = pd.read_parquet(f'_data/raw/operacoes-credito/{file}', engine='pyarrow').sample(n=sample_size, random_state=random.randint(1, 1000))
	table = pa.Table.from_pandas(temp_df)
	arrow_tables.append(table)

combined_table = pa.concat_tables(arrow_tables)

df = combined_table.to_pandas()

Processando o arquivo 201206.parquet
Processando o arquivo 201207.parquet
Processando o arquivo 201208.parquet
Processando o arquivo 201209.parquet
Processando o arquivo 201210.parquet
Processando o arquivo 201211.parquet
Processando o arquivo 201212.parquet
Processando o arquivo 201301.parquet
Processando o arquivo 201302.parquet
Processando o arquivo 201303.parquet
Processando o arquivo 201304.parquet
Processando o arquivo 201305.parquet
Processando o arquivo 201306.parquet
Processando o arquivo 201307.parquet
Processando o arquivo 201308.parquet
Processando o arquivo 201309.parquet
Processando o arquivo 201310.parquet
Processando o arquivo 201311.parquet
Processando o arquivo 201312.parquet
Processando o arquivo 201401.parquet
Processando o arquivo 201402.parquet
Processando o arquivo 201403.parquet
Processando o arquivo 201404.parquet
Processando o arquivo 201405.parquet
Processando o arquivo 201406.parquet
Processando o arquivo 201407.parquet
Processando o arquivo 201408.parquet
P

In [5]:
df.count()

data_base                         999954
uf                                999954
cliente                           999954
ocupacao                          999954
cnae_secao                        999954
porte                             999954
modalidade                        999954
numero_de_operacoes               999954
carteira_ativa                    999954
carteira_inadimplida_arrastada    999954
ativo_problematico                999954
indexador_modalidade              969533
dtype: int64

### Ajuste de tipagem dos dados

In [6]:
df.dtypes

data_base                         object
uf                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
porte                             object
modalidade                        object
numero_de_operacoes               object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
indexador_modalidade              object
dtype: object

In [7]:
df['cliente'].value_counts()

cliente
PJ    860148
PF    139806
Name: count, dtype: int64

In [8]:
df = df.rename(columns={
	'cliente': 'classificacao',
	'cnae_secao': 'cnae',
})

In [9]:
if RUN_RESOURCE_VISUALIZATION:
	print(df['porte'].value_counts())
	print(df['modalidade'].value_counts())
	print(df['ocupacao'].value_counts())
	print(df['cnae'].value_counts())

In [10]:
df['porte'] = df['porte'].str.slice(5).str.strip()
df['modalidade'] = df['modalidade'].str.slice(5).str.strip()
df['ocupacao'] = df['ocupacao'].str.slice(5).str.strip()
df['cnae'] = df['cnae'].str.slice(5).str.strip()

In [11]:
MONETARY_COLS = ['carteira_ativa', 'carteira_inadimplida_arrastada', 'ativo_problematico']

CATEGORY_COLS = ['uf', 'ocupacao', 'cnae', 'porte', 'modalidade', 'indexador_modalidade', 'classificacao']

QUANTITY_COLS = ['numero_de_operacoes']

DATE_COLS = ['data_base']

for column in df.columns:
	if column in MONETARY_COLS:
		df[column] = df[column].str.replace(',', '.').astype(float)
		df = df.rename(columns={column: f'vl_{column}'})
	elif column in CATEGORY_COLS:
		df[column] = df[column].astype('category')
		df = df.rename(columns={column: f'ct_{column}'})
	elif column in QUANTITY_COLS:
		df[column] = df[column].str.replace('<= ', '').astype(int)
		df = df.rename(columns={column: f'qt_{column}'})
	elif column in DATE_COLS:
		df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
		df = df.rename(columns={column: f'dt_{column}'})

In [12]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
qt_numero_de_operacoes                        int64
vl_carteira_ativa                           float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
dtype: object

### Exportando para zona Trusted

In [13]:
export_to_file(df, '_data/trusted/operacoes-credito')

In [14]:
df['vl_carteira_problematica'] = df['vl_ativo_problematico'] - df['vl_carteira_inadimplida_arrastada']

In [15]:
df['vl_carteira_problematica'].describe()

count       999954.00
mean        227065.97
std        7157513.60
min       -1387657.61
25%              0.00
50%              0.00
75%           5273.52
max     2850535140.13
Name: vl_carteira_problematica, dtype: float64

#### Valores menores do que 0?

In [16]:
df[df['vl_carteira_problematica'] < 0]['vl_carteira_problematica']

326042      -4018.61
362803     -24092.91
296694       -578.03
90483       -3915.97
44978       -6259.78
90542      -28169.84
11513       -2446.64
91466       -9487.79
269605     -28845.46
11975       -2325.30
159855       -949.03
91736        -275.29
91793      -31356.16
92402        -763.51
160390      -2250.67
46047       -3057.03
45776        -934.39
45825       -1039.77
232207       -744.21
61926       -9722.53
268817     -25431.42
158969      -7915.98
172645     -12008.96
416711     -26202.86
280027   -1387657.61
269962     -22663.94
342578    -657559.60
360945     -26216.65
356431      -5097.68
478354    -136508.78
275107     -64435.12
343173    -560423.89
620579      -3207.02
351660     -16132.14
242387     -19630.39
604381      -7423.77
307630      -9470.78
303072     -14559.33
353276      -1386.73
310498     -27267.92
202991       -217.35
600977    -192117.52
246722     -66666.49
166606      -5000.00
6973        -1958.17
546443     -10129.25
121681      -3694.69
81944        

#### 0.004% da amostra está danificada, vamos capar em 0

In [17]:
df['vl_carteira_problematica'] = df['vl_carteira_problematica'].clip(lower=0)

In [18]:
df['vl_carteira_ativa'] = df['vl_carteira_ativa'] - df['vl_carteira_problematica']
df.rename(columns={ 'vl_carteira_ativa': 'vl_carteira_saudavel' }, inplace=True)

In [19]:
df['vl_carteira_saudavel'].describe()

count        999954.00
mean        5392024.86
std       105438654.58
min               0.00
25%           16238.09
50%           94851.55
75%          522260.16
max     22551483154.30
Name: vl_carteira_saudavel, dtype: float64

In [20]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
qt_numero_de_operacoes                        int64
vl_carteira_saudavel                        float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
vl_carteira_problematica                    float64
dtype: object

### Exportando para zona Refined

In [21]:
export_to_file(df, '_data/refined/operacoes-credito')